<div align="center">
  <h1>Лабораторная работа №3</h1>
</div>

<div align="center">
  <h2>Отчет о выполнении</h2>
</div>

<div align="right">
  <h3>Студента 3 курса 8 группы</h3>
  <h3>Добрицкого Артема</h3>
</div>

# 1. Загрузка и предобработка данных

In [11]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

file = pd.read_csv("../data/Annual 2005-2011.csv")

# 2. Подготовка данных

In [12]:
numeric_cols = file.select_dtypes(include=[np.number]).columns
print(f"Найдено числовых колонок: {len(numeric_cols)}")
feature_columns = [col for col in numeric_cols if file[col].nunique() > 10]
print(f"Выбрано колонок для анализа: {len(feature_columns)}")

X = file[feature_columns].copy()
X = X.fillna(X.mean())
print(f"Размерность матрицы признаков: {X.shape}")

Найдено числовых колонок: 22
Выбрано колонок для анализа: 21
Размерность матрицы признаков: (2695, 21)


# 3. Кластеризация методом k-средних

In [13]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
kmeans = KMeans(n_clusters=4, random_state=42)
cluster_labels = kmeans.fit_predict(X_scaled)
cluster_centers = kmeans.cluster_centers_

file['cluster_original'] = cluster_labels

print(f"Метки кластеров: {np.unique(cluster_labels)}")

Метки кластеров: [0 1 2 3]


# 4. Перенумерация кластеров
Расчет интегрального показателя

In [14]:
X_normalized = (X - X.mean()) / X.std()
weights = np.ones(len(feature_columns)) / len(feature_columns)
file['credit_score'] = X_normalized.dot(weights)
print("Интегральный показатель кредитоспособности:")
file['credit_score']

Интегральный показатель кредитоспособности:


0      -0.154192
1      -0.062389
2      -0.538522
3       1.022190
4      -0.050420
          ...   
2690    0.153366
2691    0.012153
2692    0.306671
2693    0.041131
2694    0.315757
Name: credit_score, Length: 2695, dtype: float64

Расчитываем средние значения кредитоспособности для каждого кластера

In [15]:
cluster_means = []
for i in range(4):
    mask = file['cluster_original'] == i
    mean_score = file.loc[mask, 'credit_score'].mean()
    cluster_means.append((i, mean_score))

print("Средние значения кредитоспособности:")
for cluster_num, mean_score in cluster_means:
    print(f"Кластер {cluster_num}: {mean_score:.4f}")

Средние значения кредитоспособности:
Кластер 0: 0.5666
Кластер 1: -0.2409
Кластер 2: -0.0721
Кластер 3: 0.1737


Далее мы сортируем кластеры в зависимости от значения кредитоспособности

In [16]:
cluster_means.sort(key=lambda x: x[1], reverse=True)
cluster_mapping = {old: new for new, (old, _) in enumerate(cluster_means)}

Расчитываем средние значения кредитоспособности для каждого исходного кластера

In [17]:
cluster_stats = []
for i in range(4):
    mask = file['cluster_original'] == i
    mean_score = file.loc[mask, 'credit_score'].mean()
    count = mask.sum()
    cluster_stats.append({
        'original_cluster': i,
        'mean_credit_score': mean_score,
        'count': count
    })
    cluster_stats_sorted = sorted(cluster_stats, key=lambda x: x['mean_credit_score'], reverse=True)

for stat in cluster_stats:
    print(f"Исходный кластер {stat['original_cluster']}: среднее = {stat['mean_credit_score']:.4f}, наблюдений = {stat['count']}")

Исходный кластер 0: среднее = 0.5666, наблюдений = 229
Исходный кластер 1: среднее = -0.2409, наблюдений = 752
Исходный кластер 2: среднее = -0.0721, наблюдений = 1002
Исходный кластер 3: среднее = 0.1737, наблюдений = 712


Обновляем центры кластеров в соответствии с новой нумерацией

In [18]:
cluster_mapping = {}
for new_num, stat in enumerate(cluster_stats_sorted):
    old_num = stat['original_cluster']
    cluster_mapping[old_num] = new_num
    print(f"Новый кластер {new_num} (бывший {old_num}): среднее = {stat['mean_credit_score']:.4f}")
file['cluster'] = file['cluster_original'].map(cluster_mapping)
new_centers = np.zeros_like(cluster_centers)
for old_num, new_num in cluster_mapping.items():
    new_centers[new_num] = cluster_centers[old_num]
cluster_centers = new_centers

Новый кластер 0 (бывший 0): среднее = 0.5666
Новый кластер 1 (бывший 3): среднее = 0.1737
Новый кластер 2 (бывший 2): среднее = -0.0721
Новый кластер 3 (бывший 1): среднее = -0.2409


# 5. Результаты

In [19]:
cluster_counts = file['cluster'].value_counts().sort_index()
total_observations = len(file)

table7 = pd.DataFrame({
    'Номер кластера': cluster_counts.index,
    'Количество наблюдений': cluster_counts.values,
    'Относительная частота (%)': (cluster_counts.values / total_observations * 100).round(4)
})
display(table7.round(3), "Распределение наблюдений по кластерам")

,Номер кластера,Количество наблюдений,Относительная частота (%)
0,0,229,8.497
1,1,712,26.419
2,2,1002,37.180
3,3,752,27.904


'Распределение наблюдений по кластерам'

In [20]:
stats_list = []
for cluster_num in range(4):
    cluster_data = file[file['cluster'] == cluster_num]['credit_score']

    stats = {
        'Кластер': cluster_num,
        'Число наблюдений': len(cluster_data),
        'Среднее': cluster_data.mean().round(7),
        'Медиана': cluster_data.median().round(7),
        'Минимум': cluster_data.min().round(7),
        'Максимум': cluster_data.max().round(7),
        'Стандартное отклонение': cluster_data.std().round(7)
    }
    stats_list.append(stats)

table8 = pd.DataFrame(stats_list)
display(table8.round(3), "Описательная статистика для интегральных показателей")

,Кластер,Число наблюдений,Среднее,Медиана,Минимум,Максимум,Стандартное отклонение
0,0,229,0.567,0.521,0.027,2.345,0.350
1,1,712,0.174,0.135,-0.269,1.237,0.238
2,2,1002,-0.072,-0.089,-0.580,1.156,0.176
3,3,752,-0.241,-0.220,-1.598,0.400,0.221


'Описательная статистика для интегральных показателей'